#### KRX 종목 검색기 
- KRX에서 종목들 가져오기
- OTP가 시간별로 바뀌는 거 주의하기! 


In [7]:
# STEP 1 : 타임 구하기 
import time 
t = time.time() * 1000

# STEP 2 : Generate-OTP 
import requests 
OTP_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?bld=COM%2Ffinder_stkisu&name=form&_='+str(t)
r = requests.get(OTP_url)
r.content

b'OM+LEiF68ec3RDb2Yv7KayuatfxE1zkKqIfBoEKXXThJ4cHfAjOJxGi7Ius/71PDsq3yC5d/AD/F+HjoP5gtlf3+JgW4evfaZTkW6AXVOEbnvZc7tAMh1uxzz9DOGApi'

In [11]:
# STEP 3 : Download the list of all the listed companies 
down_url = 'http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx'
down_data = {
    'isuCd' : '' , 
    'no': 'P1' , 
    'mktsel': 'ALL' , 
    'searchText' : '' ,
    'pagePath': '/contents/COM/FinderStkIsu.jsp' ,
    'code': r.content ,
    'pageFirstCall' : 'Y'
}

down = requests.post(down_url , down_data)
down.text[:1000]

'{"block1":[{"full_code":"KR7060310000","short_code":"A060310","codeName":"3S","marketName":"KOSDAQ"},{"full_code":"KR7095570008","short_code":"A095570","codeName":"AJ네트웍스","marketName":"KOSPI"},{"full_code":"KR7068400001","short_code":"A068400","codeName":"AJ렌터카","marketName":"KOSPI"},{"full_code":"KR7006840003","short_code":"A006840","codeName":"AK홀딩스","marketName":"KOSPI"},{"full_code":"KR7054620000","short_code":"A054620","codeName":"APS홀딩스","marketName":"KOSDAQ"},{"full_code":"KR7265520007","short_code":"A265520","codeName":"AP시스템","marketName":"KOSDAQ"},{"full_code":"KR7211270004","short_code":"A211270","codeName":"AP위성","marketName":"KOSDAQ"},{"full_code":"KR7027410000","short_code":"A027410","codeName":"BGF","marketName":"KOSPI"},{"full_code":"KR7282330000","short_code":"A282330","codeName":"BGF리테일","marketName":"KOSPI"},{"full_code":"KR7138930003","short_code":"A138930","codeName":"BNK금융지주","marketName":"KOSPI"},{"full_code":"KR7001460005","short_code":"A001460","codeName":"BY

In [25]:
# json.loads 정리하기
import json
from pandas.io.json import json_normalize

json.loads(down.text)
list = json_normalize(json.loads(down.text),'block1')

In [26]:
# 함수로 정리하기 
def stock_finder() : 
    # STEP 1 : 타임 구하기 
    import time 
    t = time.time() * 1000

    # STEP 2 : Generate-OTP 
    import requests 
    OTP_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?bld=COM%2Ffinder_stkisu&name=form&_='+str(t)
    r = requests.get(OTP_url)
    r.content
    
   # STEP 3 : Download the list of all the listed companies 
    down_url = 'http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx'
    down_data = {
        'isuCd' : '' , 
        'no': 'P1' , 
        'mktsel': 'ALL' , 
        'searchText' : '' ,
        'pagePath': '/contents/COM/FinderStkIsu.jsp' ,
        'code': r.content ,
        'pageFirstCall' : 'Y'
        }

    down = requests.post(down_url , down_data)
    
    # STEP 4 : return as dataframe 
    import json
    from pandas.io.json import json_normalize
    
    return json_normalize(json.loads(down.text),'block1')

In [28]:
# stock_finder 에서 marketName 별로 grouping 해보기 
stock_finder().head(n = 10)

,codeName,full_code,marketName,short_code
0,3S,KR7060310000,KOSDAQ,A060310
1,AJ네트웍스,KR7095570008,KOSPI,A095570
2,AJ렌터카,KR7068400001,KOSPI,A068400
3,AK홀딩스,KR7006840003,KOSPI,A006840
4,APS홀딩스,KR7054620000,KOSDAQ,A054620
5,AP시스템,KR7265520007,KOSDAQ,A265520
6,AP위성,KR7211270004,KOSDAQ,A211270
7,BGF,KR7027410000,KOSPI,A027410
8,BGF리테일,KR7282330000,KOSPI,A282330
9,BNK금융지주,KR7138930003,KOSPI,A138930


In [34]:
a = stock_finder()
a.groupby('marketName').size()

marketName
KONEX      148
KOSDAQ    1274
KOSPI      923
dtype: int64

In [59]:
a_KONEX = pd.DataFrame([a.loc[x] for x in range(len(a)) if a['marketName'][x] == 'KONEX'])[['codeName' , 'full_code']].reset_index(drop = True)
a_KOSDAQ = pd.DataFrame([a.loc[x] for x in range(len(a)) if a['marketName'][x] == 'KOSDAQ'])[['codeName' , 'full_code']].reset_index(drop = True)
a_KOSPI = pd.DataFrame([a.loc[x] for x in range(len(a)) if a['marketName'][x] == 'KOSPI'])[['codeName' , 'full_code']].reset_index(drop = True)

In [63]:
print(len(a_KONEX))
a_KONEX.head(n=10) 

148


,codeName,full_code
0,KC산업,KR7112190004
1,SGA임베디드,KR7224880005
2,TS트릴리온,KR7284610003
3,관악산업,KR7076340009
4,구름게임즈앤컴퍼니,KR7223220005
5,굿센,KR7243870003
6,그린플러스,KR7186230009
7,극동자동화,KR7272420001
8,나눔테크,KR7244880001
9,나무기술,KR7167380005
